In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm  # Import tqdm for progress bar

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    
])

# Load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Convert datasets to DataLoader for batching
train_loader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

# Extract the data and labels
trainX, trainy = next(iter(train_loader))
testX, testy = next(iter(test_loader))

# Convert labels to numpy arrays (if needed)
trainy = trainy.numpy()
testy = testy.numpy()

# Function to sample a batch
def sample_batch(batch_size, device):
    # Randomly select indices
    indices = torch.randperm(trainX.shape[0])[:batch_size]
    
    # Get the batch of data
    data = trainX[indices].to(device)
    
    # Interpolate the data to 32x32
    data = F.interpolate(data, size=(32, 32), mode='bilinear', align_corners=False)
    
    return data

In [2]:
class DiffusionModel:
    def __init__(self, T: int, model: nn.Module, device: str):
        self.T = T
        self.function_approximator = model  # Corrected variable name
        self.device = device

        self.beta = torch.linspace(0.0001, 0.02, T).to(device)
        self.alpha = 1 - self.beta
        self.alpha_bar = torch.cumprod(self.alpha, dim=0)  # Corrected variable name

    def training(self, batch_size, optimizer):
        x0 = sample_batch(batch_size, self.device)  # Clean Images
        eps = torch.rand_like(x0)  # Noise

        # Corrected: Use torch.long instead of torch.long()
        t = torch.randint(0, self.T, (batch_size,), device=self.device, dtype=torch.long)  # Random timesteps

        # Take one gradient descent step
        alpha_bar_t = self.alpha_bar[t].unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)  # Corrected indexing
        eps_predicted = self.function_approximator(
            torch.sqrt(alpha_bar_t) * x0 + torch.sqrt(1 - alpha_bar_t) * eps, t)
        loss = nn.functional.mse_loss(eps, eps_predicted)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        return loss.item()

    @torch.no_grad()
    def sampling(self, n_samples=1, image_channels=1, img_size=(32, 32), use_tqdm=True):
        x = torch.randn(n_samples, image_channels, img_size[0], img_size[1], device=self.device)  # Random sample

        progress_bar = tqdm if use_tqdm else lambda x: x
        for t in progress_bar(range(self.T, 0, -1)):
            z = torch.randn_like(x) if t > 1 else torch.zeros_like(x)
            t_tensor = torch.ones(n_samples, dtype=torch.long, device=self.device) * t  # Corrected variable name

            beta_t = self.beta[t_tensor - 1].unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
            alpha_t = self.alpha[t_tensor - 1].unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
            alpha_bar_t = self.alpha_bar[t_tensor - 1].unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)

            mean = 1 / torch.sqrt(alpha_t) * (x - ((1 - alpha_t) / torch.sqrt(
                1 - alpha_bar_t)) * self.function_approximator(x, t_tensor - 1))
            sigma = torch.sqrt(beta_t)
            x = mean + sigma * z
        return x


In [4]:
# Define the UNet model (assuming it's defined in unet.py)
from unet import UNet

device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 64
model = UNet().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
diffusion_model = DiffusionModel(1000, model, device)

# Training
for epoch in tqdm(range(40000)):
    loss = diffusion_model.training(batch_size, optimizer)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss}")

  0%|          | 1/40000 [00:00<6:03:25,  1.83it/s]

Epoch 0, Loss: 0.30997204780578613


  0%|          | 101/40000 [00:29<3:09:31,  3.51it/s]

Epoch 100, Loss: 0.022970618680119514


  1%|          | 201/40000 [00:57<3:09:13,  3.51it/s]

...

100%|█████████▉| 39801/40000 [3:09:44<00:56,  3.49it/s]

Epoch 39800, Loss: 0.0009109395905397832


100%|█████████▉| 39901/40000 [3:10:12<00:28,  3.50it/s]

Epoch 39900, Loss: 0.001198838814161718


100%|██████████| 40000/40000 [3:10:40<00:00,  3.50it/s]
